In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
final_dataframe = pd.read_excel("curated_data.xlsx")

### Return some basic information about the dataframe

Results to info() shows 2537 records in total, but I can also see that there are a number of fields where the city, state, or country was not found

In [ ]:
final_dataframe.info()

describe() shows similar information, but it also tells me the unique count of business name/places.
This means I have duplication. Note to self: clean this up at a later date in the ETL code.

In [ ]:
final_dataframe.describe()

Determine unique/distinct counts for each column in the dataframe

In [ ]:
unique_counts = pd.DataFrame.from_records([(col, final_dataframe[col].nunique()) for col in final_dataframe.columns],columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

### Starting Analysis and Visualization

Search the "Note" column in final_dataframe for the string "happy hour"

In [ ]:
happy_hour_df = final_dataframe.loc[final_dataframe["Note"].str.contains("happy hour", case=False, na=False)]
happy_hour_df


I would like to eventually map frequency of places in US States. First, I will return a unique list of countries from 'Country'. This way I know how to filter for US States.

In [ ]:
# I just realized that unique and nunique returns different values for Country: 35 vs. 36. 
# Unique includes missing value (i.e. nan)
final_dataframe.Country.unique()

In [ ]:
countries = ['United States', 'US']
us_only_places_df = final_dataframe.loc[(final_dataframe['Country'].isin(countries))]
len(us_only_places_df.index)

Return a distinct list of US States
Results indicate that I have a blended list of state names (long name and short name)
This is will likely create an issue when mapping because I will join on state name

In [ ]:
us_only_places_df['State'].unique()

Now that I have a dataframe with just places in the United States, I can graph the number of "places" by US States

In [ ]:
us_map = gpd.read_file('tl_2020_us_state.shp')
names = ['Alaska', 'Hawaii', 'United States Virgin Islands', 'Commonwealth of the Northern Mariana Islands', 'Guam', 'American Samoa', 'Puerto Rico']
updated_us_map = us_map.loc[~(us_map)['NAME'].isin(names)]
state_occurrences = us_only_places_df['State'].value_counts().reset_index().rename(columns={'index':'NAME','State':'count'})
merge_map = updated_us_map.merge(state_occurrences, on='NAME')
merge_map.plot(column='count', cmap='Oranges', legend=True, legend_kwds={'label': "Number of Places", 'orientation': "vertical", 'shrink':0.4}, figsize=(12,20), edgecolor='black')
plt.title('Count of Places Saved for each US States')

I was curious if I can return the list of state names for the above shape file.
This also confirms that the shapefile is using full name for states.
If I want to further refine my data, I can go back and update all the abbreviated state names with full name.

In [ ]:
gdf = gpd.read_file("tl_2020_us_state.shp")
state_names = list(gdf["NAME"])
state_names

Graph the number of "places" by Country (i.e. the occurrences of Country in the curated dataframe)

Results here tells me that I **should** further cleanse my data so that I standardize the Country names and combine data for similar countries. For example: US and United States are the same country and **should** be combined under "United States". Further research required to return long name for: MX, PR, and CA.

In [ ]:
country_occurrences = final_dataframe['Country'].value_counts()
# colors = ['#' + ''.join(random.choices('0123456789ABCDEF', k=6)) for _ in range(len(country_occurrences))]
plt.figure(figsize=(20, 12))
plt.barh(country_occurrences.index, country_occurrences.values, color = "g")
plt.xlabel("Occurrences")
plt.ylabel("Country")
plt.title("Count of Saved Places by Country")
plt.xticks(range(0,max(country_occurrences)+1, 50))

for i, v in enumerate(country_occurrences.values):
    plt.text(v + 10, i, str(v), color='black', fontweight='bold')
